# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to /Users/karol/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/karol/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/karol/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [88]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy
import re
from time import time

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import classification_report

import pickle

In [2]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages', con=engine)
X = df['message'].values
y = df.drop(columns=['id', 'message', 'original', 'genre']).values

In [3]:
print(X,y)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    # tokenize text
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(re.sub(r"[^a-zA-Z0-9]", " ", tok.lower())).strip() 
        clean_tokens.append(clean_tok)

    return clean_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
pipeline.fit(X_train,y_train)
# predict on test data
y_pred = pipeline.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [32]:
labels = df.drop(columns=['id', 'message', 'original', 'genre']).columns
#print("y {} \n shape {} \ny_pred {} \n shape {}".format(y_test,y_test.shape,y_pred,y_pred.shape))
print(classification_report(y_test, y_pred, target_names=labels))

#y.min().max()

             precision    recall  f1-score   support

          0       0.84      0.92      0.88      4981
          1       0.79      0.45      0.57      1104

avg / total       0.74      0.47      0.53     20622



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [41]:
parameters = {
    'vect__max_features': (None,5000),
    'tfidf__use_idf': (True,False),
    'clf__estimator__n_estimators': [10,20,50],
    'clf__estimator__min_samples_split': [2,3,4]
}

cv = GridSearchCV(estimator=pipeline,param_grid=parameters, n_jobs=-1, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [42]:
#Training with grid search
t0 = time()
cv.fit(X_train,y_train)
print("done in %0.3fs" % (time() - t0))

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_features': (None, 5000), 'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20, 50], 'clf__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [85]:
#Testing model
y_pred = cv.predict(X_test)

In [87]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50, 'tfidf__use_idf': True, 'vect__max_features': 5000}


In [86]:
labels = df.drop(columns=['id', 'message', 'original', 'genre']).columns.tolist()
#print(labels)
#print("y_test {} \n shape {} type: {} \ny_pred {} \n shape {} type {}".format(y_test,y_test.shape,type(y_test),y_pred,y_pred.shape, type(y_pred)))
print(classification_report(y_test, y_pred, target_names=labels))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      4981
               request       0.82      0.50      0.63      1104
                 offer       0.00      0.00      0.00        24
           aid_related       0.75      0.70      0.72      2708
          medical_help       0.62      0.12      0.20       509
      medical_products       0.76      0.16      0.26       348
     search_and_rescue       0.71      0.13      0.23       187
              security       0.25      0.01      0.02       117
              military       0.55      0.07      0.13       227
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.43      0.57       440
                  food       0.82      0.67      0.74       719
               shelter       0.82      0.47      0.60       580
              clothing       0.87      0.12      0.22       104
                 money       0.57      

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [83]:
#Trying with different algorithms

def build_model(estimator):
    # text processing and model pipeline
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator=estimator))
    ])

    # define parameters for GridSearchCV
    parameters = {
    'vect__max_features': (None,5000),
    'tfidf__use_idf': (True,False),
    #'clf__estimator__penalty': ['none', 'l2', 'elasticnet'],
    #'clf__estimator__loss': ['hinge', 'log','perceptron']
    }

     # create gridsearch object and return as final model pipeline
    cv = GridSearchCV(estimator=pipeline,param_grid=parameters, n_jobs=-1, 
                  verbose=2)

   
    return cv

In [73]:
model = build_model(KNeighborsClassifier())
#Training with grid search
t0 = time()

model.fit(X_train,y_train)

#print(np.unique(y_train))
print("done in %0.3fs" % (time() - t0))

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] tfidf__use_idf=True, vect__max_features=None ....................
[CV] tfidf__use_idf=True, vect__max_features=None ....................
[CV] tfidf__use_idf=True, vect__max_features=None ....................
[CV] tfidf__use_idf=True, vect__max_features=5000 ....................
[CV] ..... tfidf__use_idf=True, vect__max_features=None, total= 4.5min
[CV] tfidf__use_idf=True, vect__max_features=5000 ....................
[CV] ..... tfidf__use_idf=True, vect__max_features=5000, total= 4.6min
[CV] tfidf__use_idf=True, vect__max_features=5000 ....................
[CV] ..... tfidf__use_idf=True, vect__max_features=None, total= 4.6min
[CV] tfidf__use_idf=True, vect__max_features=10000 ...................
[CV] ..... tfidf__use_idf=True, vect__max_features=None, total= 4.6min
[CV] tfidf__use_idf=True, vect__max_features=10000 ...................
[CV] ..... tfidf__use_idf=True, vect__max_features=5000, total= 4.6min
[CV] tfidf__use_i

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 52.9min finished


done in 3258.616s


In [76]:
y_pred = model.predict(X_test)

In [77]:
print("\nBest Parameters:", model.best_params_)
labels = df.drop(columns=['id', 'message', 'original', 'genre']).columns.tolist()
#print(labels)
#print("y_test {} \n shape {} type: {} \ny_pred {} \n shape {} type {}".format(y_test,y_test.shape,type(y_test),y_pred,y_pred.shape, type(y_pred)))
print(classification_report(y_test, y_pred, target_names=labels))


Best Parameters: {'tfidf__use_idf': True, 'vect__max_features': 10000}
                        precision    recall  f1-score   support

               related       0.79      0.74      0.76      4981
               request       0.78      0.12      0.20      1104
                 offer       0.00      0.00      0.00        24
           aid_related       0.75      0.05      0.10      2708
          medical_help       0.67      0.00      0.01       509
      medical_products       0.67      0.02      0.03       348
     search_and_rescue       0.00      0.00      0.00       187
              security       0.00      0.00      0.00       117
              military       0.00      0.00      0.00       227
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.06      0.11       440
                  food       0.77      0.09      0.17       719
               shelter       0.82      0.06      0.11       580
              clothing       0.

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [84]:
#Was taking more thatn 8 hours, so I interrupted it.
model = build_model(MLPClassifier())
model.fit(X_train,y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] tfidf__use_idf=True, vect__max_features=None ....................
[CV] tfidf__use_idf=True, vect__max_features=None ....................
[CV] tfidf__use_idf=True, vect__max_features=None ....................
[CV] tfidf__use_idf=True, vect__max_features=5000 ....................


/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

### 9. Export your model as a pickle file

In [89]:
filename = '../models/final_model.pkl'
pickle.dump(cv,open(filename,'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.